In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import collections
%matplotlib inline
import os

In [0]:
# Imports and functions from 5' Model
%matplotlib inline
from matplotlib.colors import LogNorm
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,Input, Flatten
from keras import models
from keras import optimizers
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
nuc_arr = ['A','C','G','T']
#Function for calculating modified probability of splicing at SD1
def prob_SD1 (sd1_freq, sd2_freq): 
    if (sd1_freq==0 and sd2_freq==0):
        return 0.0 
    else:
        return sd1_freq/(sd1_freq+sd2_freq)
#Function converting nucleotide sequence to numerical array with 4 channels
def seq_to_arr (seq):
    seq_len = len(seq)
    arr_rep = np.zeros((seq_len, len(nuc_arr))) 
    for i in range(seq_len):
        arr_rep[i][nuc_arr.index(seq[i])] = 1 
    return arr_rep

In [0]:
#Creating a modified dataset with only the necessary information
#Storing model inputs and outputs
reads_path = "/content/gdrive/My Drive/Active Learning Research 2019/GSM1911086_A5SS_spliced_reads.txt"
seq_path = "/content/gdrive/My Drive/Active Learning Research 2019/GSM1911085_A5SS_seq.txt"
s1_indx = 1
s2_indx= 45
seq_len = 101
read_lines = []
seq_lines = []
data_table = []
with open(reads_path) as f:
    f.readline() 
    for line in f:
        mod_line = line.split('\t')
        read_lines.append([mod_line[0], mod_line[s1_indx], mod_line[s2_indx]])
with open(seq_path) as f:
    f.readline() 
    for line in f:
        mod_line = line.split('\t') 
        seq_lines.append([mod_line[0], mod_line[1][:-1]])
        
n = len(read_lines)
prob_s1 = np.zeros(n)
inputs = np.zeros((n,seq_len, 4))

for i in range(n):
    prob_s1[i] = prob_SD1(float(read_lines[i][1]), float(read_lines[i][2]))
    inputs[i] = seq_to_arr(seq_lines[i][1])

In [0]:
# The CNP takes as input a `CNPRegressionDescription` namedtuple with fields:
#   `query`: a tuple containing ((context_x, context_y), target_x)
#   `target_y`: a tesor containing the ground truth for the targets to be
#     predicted
#   `num_total_points`: A vector containing a scalar that describes the total
#     number of datapoints used (context + target)
#   `num_context_points`: A vector containing a scalar that describes the number
#     of datapoints used as context
# The GPCurvesReader returns the newly sampled data in this format at each
# iteration

CNPRegressionDescription = collections.namedtuple(
    "CNPRegressionDescription",
    ("query", "target_y", "num_total_points", "num_context_points"))


class GPCurvesReader(object):

    def __init__(self,
                batch_size,
                max_num_context,
                x_train,
                x_test,
                y_train,
                y_test,
                x_size=404,
                y_size=1,
                l1_scale=0.4,
                sigma_scale=1.0,
                testing=False):

        self._batch_size = batch_size
        self._max_num_context = max_num_context
        self._x_size = x_size
        self._y_size = y_size
        self._x_train = x_train
        self._x_test = x_test
        self._y_train = y_train
        self._y_test = y_test
        self._l1_scale = l1_scale
        self._sigma_scale = sigma_scale
        self._testing = testing

    def _gaussian_kernel(self, xdata, l1, sigma_f, sigma_noise=2e-2):

        num_total_points = tf.shape(xdata)[1]

        # Expand and take the difference
        xdata1 = tf.expand_dims(xdata, axis=1)  # [B, 1, num_total_points, x_size]
        xdata2 = tf.expand_dims(xdata, axis=2)  # [B, num_total_points, 1, x_size]
        diff = xdata1 - xdata2  # [B, num_total_points, num_total_points, x_size]

        # [B, y_size, num_total_points, num_total_points, x_size]
        norm = tf.square(diff[:, None, :, :, :] / l1[:, :, None, None, :])

        norm = tf.reduce_sum(
            norm, -1)  # [B, data_size, num_total_points, num_total_points]

        # [B, y_size, num_total_points, num_total_points]
        kernel = tf.square(sigma_f)[:, :, None, None] * tf.exp(-0.5 * norm)

        # Add some noise to the diagonal to make the cholesky work.
        kernel += (sigma_noise**2) * tf.eye(num_total_points)

        return kernel
    def generate_curves(self):
        """Builds the op delivering the data.

        Generated functions are `float32` with x values between -2 and 2.

        Returns:
          A `CNPRegressionDescription` namedtuple.
        """
        
        #print(num_context)
        # If we are testing we want to have more targets and have them evenly
        # distributed in order to plot the function.
        # When testing, we want context points randomly sampled from training data
        # and target points randomly sampled from testing data
        if self._testing:
            num_testing_points = tf.Tensor.get_shape(self._y_test)[0]
            num_training_points = tf.Tensor.get_shape(self._y_train)[0]
            num_context = num_training_points
            # Select the targets (all training data)
            indices_test = np.arange(int(num_testing_points))
            target_x_values = [tf.gather(self._x_test, tf.convert_to_tensor(indices_test))]
            target_y_values = [tf.gather(self._y_test, tf.convert_to_tensor(indices_test))]
            target_x = tf.stack(target_x_values)
            target_y_values = tf.stack(target_y_values)
            target_y = target_y_values
            #target_y = tf.expand_dims(target_y_values, axis=2)
            
            # Select the observations (all testing data)
            indices_train = np.arange(int(num_training_points))
            context_x_values = [tf.gather(self._x_train, tf.convert_to_tensor(indices_train))]
            context_y_values = [tf.gather(self._y_train, tf.convert_to_tensor(indices_train))]
            context_x = tf.stack(context_x_values)
            context_y_values = tf.stack(context_y_values)
            context_y = context_y_values
            #context_y = tf.expand_dims(context_y_values, axis=2)
            
            
            #context_x = tf.gather(x_values, idx[:int(num_context)], axis=1)
            #context_y = tf.gather(y_values, idx[:int(num_context)], axis=1)
        # During training the number of target points and their x-positions are
        # selected at random
        else:
            num_context = self._max_num_context
            num_target = self._max_num_context
            num_total_points = int(num_context + num_target)
            x_values = []
            y_values = []
            num_training_points = tf.Tensor.get_shape(self._y_train)[0]
            #print(num_context)
            #print(num_target)
            #print(num_total_points)
            for batch_num in range(self._batch_size):
                indices = np.random.choice(num_training_points,num_total_points,replace=False)
                x_values_batch = tf.gather(self._x_train, tf.convert_to_tensor(indices))
                y_values_batch = tf.gather(self._y_train, tf.convert_to_tensor(indices))
                x_values.append(x_values_batch)
                y_values.append(y_values_batch)
            x_values = tf.stack(x_values)
            y_values = tf.stack(y_values)
            #y_values = tf.expand_dims(y_values, axis=2)
            #print(tf.shape(y_values))
            #x_values = tf.random_uniform(
            #  [self._batch_size, num_total_points, self._x_size], -2, 2)
            # Select the targets which will consist of the context points as well as
            # some new target points
            target_x = x_values[:, :num_total_points, :]
            target_y = y_values[:, :num_total_points, :]
            # Select the observations
            context_x = x_values[:, :int(num_context), :]
            context_y = y_values[:, :int(num_context), :]

        query = ((context_x, context_y), target_x)
        return CNPRegressionDescription(
            query=query,
            target_y=target_y,
            num_total_points=tf.shape(target_x)[1],
            num_context_points=num_context)

In [0]:
class DeterministicEncoder(object):
  """The Encoder."""

  def __init__(self, output_sizes):
    """CNP encoder.

    Args:
      output_sizes: An iterable containing the output sizes of the encoding MLP.
    """
    self._output_sizes = output_sizes

  def __call__(self, context_x, context_y, num_context_points):
    """Encodes the inputs into one representation.

    Args:
      context_x: Tensor of size bs x observations x m_ch. For this 1D regression
          task this corresponds to the x-values.
      context_y: Tensor of size bs x observations x d_ch. For this 1D regression
          task this corresponds to the y-values.
      num_context_points: A tensor containing a single scalar that indicates the
          number of context_points provided in this iteration.

    Returns:
      representation: The encoded representation averaged over all context 
          points.
    """
    # Concatenate x and y along the filter axes
    encoder_input = tf.concat([context_x, context_y], axis=-1)
    # Get the shapes of the input and reshape to parallelise across observations
    batch_size, _, filter_size = encoder_input.shape.as_list()
    hidden = tf.reshape(encoder_input, (batch_size * int(num_context_points), -1))
    hidden.set_shape((None, filter_size))
    
    # Pass through MLP
    with tf.variable_scope("encoder", reuse=tf.AUTO_REUSE):
      for i, size in enumerate(self._output_sizes[:-1]):
        hidden = tf.nn.relu(
            tf.layers.dense(hidden, size, name="Encoder_layer_{}".format(i)))

      # Last layer without a ReLu
      hidden = tf.layers.dense(
          hidden, self._output_sizes[-1], name="Encoder_layer_{}".format(i + 1))

    # Bring back into original shape
    hidden = tf.reshape(hidden, (batch_size, int(num_context_points), size))

    # Aggregator: take the mean over all points
    representation = tf.reduce_mean(hidden, axis=1)
    
    return representation

In [0]:
class DeterministicDecoder(object):
  """The Decoder."""

  def __init__(self, output_sizes):
    """CNP decoder.

    Args:
      output_sizes: An iterable containing the output sizes of the decoder MLP 
          as defined in `basic.Linear`.
    """
    self._output_sizes = output_sizes

  def __call__(self, representation, target_x, num_total_points):
    """Decodes the individual targets.

    Args:
      representation: The encoded representation of the context
      target_x: The x locations for the target query
      num_total_points: The number of target points.

    Returns:
      dist: A multivariate Gaussian over the target points.
      mu: The mean of the multivariate Gaussian.
      sigma: The standard deviation of the multivariate Gaussian.
    """

    # Concatenate the representation and the target_x
    representation = tf.tile(
        tf.expand_dims(representation, axis=1), [1, num_total_points, 1])
    input = tf.concat([representation, target_x], axis=-1)

    # Get the shapes of the input and reshape to parallelise across observations
    batch_size, _, filter_size = input.shape.as_list()
    hidden = tf.reshape(input, (batch_size * num_total_points, -1))
    hidden.set_shape((None, filter_size))

    # Pass through MLP
    with tf.variable_scope("decoder", reuse=tf.AUTO_REUSE):
      for i, size in enumerate(self._output_sizes[:-1]):
        hidden = tf.nn.relu(
            tf.layers.dense(hidden, size, name="Decoder_layer_{}".format(i)))

      # Last layer without a ReLu
      hidden = tf.layers.dense(
          hidden, self._output_sizes[-1], name="Decoder_layer_{}".format(i + 1))

    # Bring back into original shape
    hidden = tf.reshape(hidden, (batch_size, num_total_points, -1))

    # Get the mean an the variance
    mu, log_sigma = tf.split(hidden, 2, axis=-1)

    # Bound the variance
    sigma = 0.1 + 0.9 * tf.nn.softplus(log_sigma)

    # Get the distribution
    dist = tf.contrib.distributions.MultivariateNormalDiag(
        loc=mu, scale_diag=sigma)

    return dist, mu, sigma

In [0]:
class DeterministicModel(object):
  """The CNP model."""

  def __init__(self, encoder_output_sizes, decoder_output_sizes):
    """Initialises the model.

    Args:
      encoder_output_sizes: An iterable containing the sizes of hidden layers of
          the encoder. The last one is the size of the representation r.
      decoder_output_sizes: An iterable containing the sizes of hidden layers of
          the decoder. The last element should correspond to the dimension of
          the y * 2 (it encodes both mean and variance concatenated)
    """
    self._encoder = DeterministicEncoder(encoder_output_sizes)
    self._decoder = DeterministicDecoder(decoder_output_sizes)

  def __call__(self, query, num_total_points, num_contexts, target_y=None):
    """Returns the predicted mean and variance at the target points.

    Args:
      query: Array containing ((context_x, context_y), target_x) where:
          context_x: Array of shape batch_size x num_context x 1 contains the 
              x values of the context points.
          context_y: Array of shape batch_size x num_context x 1 contains the 
              y values of the context points.
          target_x: Array of shape batch_size x num_target x 1 contains the
              x values of the target points.
      target_y: The ground truth y values of the target y. An array of 
          shape batchsize x num_targets x 1.
      num_total_points: Number of target points.

    Returns:
      log_p: The log_probability of the target_y given the predicted
      distribution.
      mu: The mean of the predicted distribution.
      sigma: The variance of the predicted distribution.
    """

    (context_x, context_y), target_x = query

    # Pass query through the encoder and the decoder
    representation = self._encoder(context_x, context_y, num_contexts)
    dist, mu, sigma = self._decoder(representation, target_x, num_total_points)

    # If we want to calculate the log_prob for training we will make use of the
    # target_y. At test time the target_y is not available so we return None
    if target_y is not None:
      log_p = dist.log_prob(target_y)
    else:
      log_p = None

    return log_p, mu, sigma

In [0]:
def plot_functions(target_x, target_y, context_x, context_y, pred_y, var):
  """Plots the predicted mean and variance and the context points.
  
  Args: 
    target_x: An array of shape batchsize x number_targets x 1 that contains the
        x values of the target points.
    target_y: An array of shape batchsize x number_targets x 1 that contains the
        y values of the target points.
    context_x: An array of shape batchsize x number_context x 1 that contains 
        the x values of the context points.
    context_y: An array of shape batchsize x number_context x 1 that contains 
        the y values of the context points.
    pred_y: An array of shape batchsize x number_targets x 1  that contains the
        predicted means of the y values at the target points in target_x.
    pred_y: An array of shape batchsize x number_targets x 1  that contains the
        predicted variance of the y values at the target points in target_x.
  """
  # Plot everything
  plt.plot(target_x[0], pred_y[0], 'b', linewidth=2)
  plt.plot(target_x[0], target_y[0], 'k:', linewidth=2)
  plt.plot(context_x[0], context_y[0], 'ko', markersize=10)
  plt.fill_between(
      target_x[0, :, 0],
      pred_y[0, :, 0] - var[0, :, 0],
      pred_y[0, :, 0] + var[0, :, 0],
      alpha=0.2,
      facecolor='#65c9f7',
      interpolate=True)

  # Make the plot pretty
  plt.yticks([-2, 0, 2], fontsize=16)
  plt.xticks([-2, 0, 2], fontsize=16)
  plt.ylim([-2, 2])
  plt.grid(False)
  ax = plt.gca()
  ax.set_facecolor('white')
  plt.show()


In [0]:
#Resetting Graphs
tf.reset_default_graph()

In [27]:
#Randomized Train-Test Split
train_indices_1 = np.load("/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/trainindices1.npy")
test_indices_1 = np.load("/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/testindices1.npy")
X_train_1 = inputs[train_indices_1]
X_test_1 = inputs[test_indices_1]
y_train_1 = prob_s1[train_indices_1]
y_test_1 = prob_s1[test_indices_1]
#X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(inputs, prob_s1,test_size=0.1, random_state=40) 
#Compressing each input X to 1D for dimensional consistency with MLP Encoder
X_train_lst = np.array([data_point.flatten() for data_point in X_train_1])
X_train_1 = tf.constant(X_train_lst)
X_test_lst = np.array([test_point.flatten() for test_point in X_test_1])
X_test_1 = tf.constant(X_test_lst)
y_train_lst = np.array([[training_y] for training_y in y_train_1])
y_train_1 = tf.constant(y_train_lst)
y_test_1 = tf.convert_to_tensor(y_test_1, dtype=tf.float64)
#y_test_1 = tf.constant(y_test_1)
print(tf.shape(X_train_1))
print(tf.shape(y_test_1))

Tensor("Shape:0", shape=(2,), dtype=int32)
Tensor("Shape_1:0", shape=(1,), dtype=int32)


In [0]:
#Train and Test Datasets
TRAINING_ITERATIONS=3000
PRINT_AFTER=1
#TRAINING_ITERATIONS = int(2e4)
MAX_CONTEXT_POINTS = 2000

dataset_train_5_prime = GPCurvesReader(
    batch_size=4, x_train=X_train_1, x_test=X_test_1, y_train=y_train_1, y_test=y_test_1, max_num_context=MAX_CONTEXT_POINTS)
data_train = dataset_train_5_prime.generate_curves()
dataset_test_5_prime = GPCurvesReader(
    batch_size=1, x_train=X_train_1, x_test=X_test_1, y_train=y_train_1, y_test=y_test_1, max_num_context=MAX_CONTEXT_POINTS,testing=True)
data_test = dataset_test_5_prime.generate_curves()


In [29]:
# Sizes of the layers of the MLPs for the encoder and decoder
# The final output layer of the decoder outputs two values, one for the mean and
# one for the variance of the prediction at the target location
encoder_output_sizes = [128, 128, 128, 128]
decoder_output_sizes = [128, 128, 2]

# Define the model
model = DeterministicModel(encoder_output_sizes, decoder_output_sizes)

# Define the loss
log_prob, _, _ = model(data_train.query, data_train.num_total_points,
                       data_train.num_context_points, data_train.target_y)

loss = -tf.reduce_mean(log_prob)

# Get the predicted mean and variance at the target points for the testing set
_, mu, sigma = model(data_test.query, data_test.num_total_points,
                     data_test.num_context_points)


# Set up the optimizer and train step
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(loss)
init = tf.initialize_all_variables()

with tf.Session() as sess:
  sess.run(init)

  for it in range(TRAINING_ITERATIONS):
    sess.run([train_step])

    # Plot the predictions in `PLOT_AFTER` intervals
    if it % PRINT_AFTER == 0:
      loss_value, pred_y, var, target_y, whole_query = sess.run(
          [loss, mu, sigma, data_test.target_y, data_test.query])

      (context_x, context_y), target_x = whole_query
      print('Iteration: {}, loss: {}'.format(it, loss_value))


Iteration: 0, loss: 0.8734886079766493
Iteration: 1, loss: 0.8244736921168431
Iteration: 2, loss: 0.7822820476889528
Iteration: 3, loss: 0.747035132305987
Iteration: 4, loss: 0.7185566531015679
Iteration: 5, loss: 0.6963092859427287
Iteration: 6, loss: 0.6792776411133825
Iteration: 7, loss: 0.6660143883818367
Iteration: 8, loss: 0.6547609196155764
Iteration: 9, loss: 0.6437029612216816
Iteration: 10, loss: 0.6313916741548378
Iteration: 11, loss: 0.6170383301830598
Iteration: 12, loss: 0.6006567402281943
Iteration: 13, loss: 0.5829266549789308
Iteration: 14, loss: 0.5650327317530437
Iteration: 15, loss: 0.5483377023954064
Iteration: 16, loss: 0.5340727644168454
Iteration: 17, loss: 0.5229501106401256
Iteration: 18, loss: 0.514948365846081
Iteration: 19, loss: 0.5091915827626338
Iteration: 20, loss: 0.5043267136700557
Iteration: 21, loss: 0.49909933032038845
Iteration: 22, loss: 0.49300916717215204
Iteration: 23, loss: 0.48661883285332125
Iteration: 24, loss: 0.48124913048527995
Iteratio

In [0]:
_, mu, sigma = model(data_test.query, data_test.num_total_points,
                     data_test.num_context_points)
sess = tf.InteractiveSession()
tf.initializers.global_variables().run()
predictions = mu[0,:,0].eval()
true_y = y_test_1.eval()
sess.close()

In [31]:
#r2 of splicing predictions on test dataset
print('predictions: ' + str(predictions))
print('true y values: ' + str(true_y))
print('mean squared error: ' + str(mean_squared_error(true_y, predictions)))
print('r2: ' + str(r2_score(true_y, predictions)))

predictions: [0.1917588  0.17465036 0.40791608 ... 0.35786017 0.47494458 0.27357091]
true y values: [1.         1.         0.75       ... 0.97727273 0.05882353 0.        ]
mean squared error: 0.19148582632827937
r2: -0.34211512959374346


In [32]:
#Calculating MSE when predicting all 0's as a Benchmark
test_zeroes = np.zeros(predictions.shape[0])
print('mean squared error when predicting exclusively 0: ' + str(mean_squared_error(true_y, test_zeroes)))

mean squared error when predicting exclusively 0: 0.24100631597304542
